In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os

def preprocess_tiff_images(input_folder, output_folder, new_width, new_height):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through each TIFF file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".tif") or filename.endswith(".tiff"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Read the TIFF image
            image = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)

            # Perform preprocessing steps
            # Example: Enhance image clarity during resizing
            resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

            # Example: Apply bilateral filter for noise reduction without blurring edges
            filtered_image = cv2.bilateralFilter(resized_image, d=9, sigmaColor=75, sigmaSpace=75)

            # Save the preprocessed image to the output folder
            cv2.imwrite(output_path, filtered_image)

# Define input and output folders
input_folder_path = '/content/drive/MyDrive/ad/augmented'
output_folder_path = '/content/drive/MyDrive/ad/Sweety'
# Define the new width and height for resizing
new_width = 256
new_height = 256

# Call the preprocessing function
preprocess_tiff_images(input_folder_path, output_folder_path, new_width, new_height)


In [ ]:

import os
import cv2
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Path to the folder containing TIFF images
data_path = "/content/drive/MyDrive/ad/augmented"

# Path to the CSV file with labels (assuming it has 'Image_path' and 'label' columns)
csv_path = "/content/drive/MyDrive/ad/output_agumented.csv"

# Load labels from CSV file
df = pd.read_csv(csv_path)
# Lists to store images and labels
images = []
labels = []
# Iterate thrdfw in df.iterrows():
img_path =df["Image_path"].tolist()
print(img_path)
lab=df["Labels"]
newlab=lab.tolist()

images = np.array(img_path)
labels = np.array(newlab)

# Normalize pixel values

# Reshape input images
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
# Print the shapes of the datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

['/content/drive/MyDrive/ad/augmented/subject13_middle_augmented_0_5535.tiff', '/content/drive/MyDrive/ad/augmented/subject13_middle_augmented_0_8128.tiff', '/content/drive/MyDrive/ad/augmented/subject13_middle_augmented_0_8719.tiff', '/content/drive/MyDrive/ad/augmented/subject13_middle_augmented_0_6449.tiff', '/content/drive/MyDrive/ad/augmented/subject13_middle_augmented_0_4518.tiff', '/content/drive/MyDrive/ad/augmented/subject13_middle_augmented_0_5709.tiff', '/content/drive/MyDrive/ad/augmented/subject13_middle_augmented_0_2320.tiff', '/content/drive/MyDrive/ad/augmented/subject13_middle_augmented_0_7900.tiff', '/content/drive/MyDrive/ad/augmented/subject13_middle_augmented_0_7894.tiff', '/content/drive/MyDrive/ad/augmented/subject13_middle_augmented_0_2115.tiff', '/content/drive/MyDrive/ad/augmented/subject21_bottom_augmented_0_6716.tiff', '/content/drive/MyDrive/ad/augmented/subject21_bottom_augmented_0_3954.tiff', '/content/drive/MyDrive/ad/augmented/subject21_bottom_augmented

In [ ]:


import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import LabelEncoder

# Load CSV file
csv_path = "/content/drive/MyDrive/ad/output_agumented.csv"
df = pd.read_csv(csv_path)
print(df)

# Encode class labels
y_cols = ['Labels']
df['classes'] = df[y_cols].apply(lambda x: x.tolist(), axis=1)
print(df)

# Define image and batch size
img_size = (224, 224)
batch_size = 1255

# Create ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split the data into training and validation sets
)

# Create data generators for training, validation, and test
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col='Image_path',
    y_col='classes',
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    class_mode='categorical',  # Assuming you have converted labels to one-hot encoding
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col='Image_path',
    y_col='classes',
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical',
)

# Build DenseNet201 model
base_model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))  # Assuming 4 classes

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=100  # Adjust as needed
)

# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=df,
    x_col='Image_path',
    y_col='classes',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Important for evaluation
)

# Predictions
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Calculate F1 score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'Weighted F1 score: {f1}')

# Classification Report
class_labels = list(test_generator.class_indices.keys())

# Convert class labels to strings
class_labels_str = [str(label) for label in class_labels]

# Classification Report
print(classification_report(y_true, y_pred_classes, target_names=class_labels_str))


                                             Image_path Labels  Severity
0     /content/drive/MyDrive/ad/augmented/subject13_...    CLE         3
1     /content/drive/MyDrive/ad/augmented/subject13_...    CLE         3
2     /content/drive/MyDrive/ad/augmented/subject13_...    CLE         3
3     /content/drive/MyDrive/ad/augmented/subject13_...    CLE         3
4     /content/drive/MyDrive/ad/augmented/subject13_...    CLE         3
...                                                 ...    ...       ...
1249  /content/drive/MyDrive/ad/augmented/subject11_...    PSE         5
1250  /content/drive/MyDrive/ad/augmented/subject11_...    PSE         5
1251  /content/drive/MyDrive/ad/augmented/subject11_...    PSE         5
1252  /content/drive/MyDrive/ad/augmented/subject11_...    PSE         5
1253  /content/drive/MyDrive/ad/augmented/subject13_...    CLE         3

[1254 rows x 3 columns]
                                             Image_path Labels  Severity  \
0     /content/drive/My

In [ ]:
print(f"Number of images: {len(sample_images)}")

Number of images: 1254


In [ ]:
import matplotlib.pyplot as plt

# Function to display images with predictions
def display_images_with_predictions(images, true_labels, predicted_labels, class_labels, num_images=16):
    num_rows = (num_images + 4) // 5
    plt.figure(figsize=(25, 5 * num_rows))
    for i in range(num_images):
        plt.subplot(num_rows, 5, i + 1)
        plt.imshow(images[i])
        plt.title(f'True: {class_labels[true_labels[i]]}\nPredicted: {class_labels[predicted_labels[i]]}')
        plt.axis('off')
    plt.show()

# Load the images for display
sample_images, sample_labels = next(test_generator)

# Make predictions on the sample images
sample_predictions = model.predict(sample_images)
sample_pred_classes = np.argmax(sample_predictions, axis=1)

# Display the images along with predictions
display_images_with_predictions(sample_images, np.argmax(sample_labels, axis=1), sample_pred_classes, class_labels, num_images=len(sample_images))


40/40 [==============================] - 5s 138ms/step


ValueError: Image size of 2500x125500 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 2500x125500 with 1254 Axes>